In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv', index_col='id')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv', index_col='id')
df_submission = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

In [3]:
df_train.drop('Policy Start Date', axis=1, inplace=True)
df_test.drop('Policy Start Date', axis=1, inplace=True)

In [4]:
target = 'Premium Amount'

In [5]:
cat = df_train.select_dtypes(include=['object']).columns

In [6]:
X = df_train.copy()
X.drop(target, axis=1, inplace=True)
X[cat] = X[cat].astype('category')
y = df_train[target]
y = np.log1p(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=45682)

In [8]:
model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True, n_estimators=100)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [9]:
df_test[cat] = df_test[cat].astype('category')
predict = model.predict(df_test)
predict = np.expm1(predict)
predict

array([765.4569 , 936.72723, 778.13885, ..., 744.21356, 731.6571 ,
       760.84503], dtype=float32)

In [10]:
df_submission[target] = predict
df_submission.head()

,id,Premium Amount
0,1200000,765.456909
1,1200001,936.727234
2,1200002,778.138855
3,1200003,862.501709
4,1200004,783.257324


In [11]:
df_submission.to_csv('submission.csv', index=False)